In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

# ===============================
# 1. 파일 경로
# ===============================
input_path = "./data/진짜 혼자 하기 싫음.xlsx"
output_path = "./data/output_filled_2.xlsx"

# ===============================
# 2. 기준 연도
# ===============================
years = [2019, 2020, 2021, 2022, 2023, 2024]

# ===============================
# 3. 엑셀 파일 로드
# ===============================
xls = pd.ExcelFile(input_path)

# ===============================
# 4. 결과 저장용 writer
# ===============================
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:

    # 🔁 시트 하나씩 반복
    for sheet_name in xls.sheet_names:

        df = pd.read_excel(xls, sheet_name=sheet_name)

        # ------------------------------------------------
        # A. 필수 컬럼 없으면 그대로 저장
        # ------------------------------------------------
        if not {"기업명", "연도"}.issubset(df.columns):
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            continue

        # ------------------------------------------------
        # B. 기업명 + 연도 중복 제거 (🔥 핵심 수정)
        # ------------------------------------------------
        df = (
            df
            .sort_values(['기업명', '연도'])
            .drop_duplicates(subset=['기업명', '연도'], keep='first')
            .reset_index(drop=True)
        )

        # ------------------------------------------------
        # C. 기업 × 연도 전체 조합 생성
        # ------------------------------------------------
        base = (
            df[['기업명']]
            .drop_duplicates()
            .assign(key=1)
            .merge(
                pd.DataFrame({'연도': years, 'key': 1}),
                on='key'
            )
            .drop('key', axis=1)
        )

        # ------------------------------------------------
        # D. 연도 보정 병합
        # ------------------------------------------------
        df_filled = (
            base
            .merge(df, on=['기업명', '연도'], how='left')
            .sort_values(['기업명', '연도'])
            .reset_index(drop=True)
        )

        # ------------------------------------------------
        # E. 시트 저장
        # ------------------------------------------------
        df_filled.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ 연도 보정 + 중복 제거 완료 (기업×연도 1행 보장)")


In [ ]:
input_path = "./data/output_filled.xlsx"
output_path = "./data/output_filled_2.xlsx"

